In [461]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import requests
import subprocess
import time
import socket
import voila

In [462]:
base_url ='https://data-brigade1-382dfaa08fc0.herokuapp.com/data-brigade1'

In [463]:

username_input = widgets.Text(placeholder='Username')
password_input = widgets.Password(placeholder='Password')

login_button = widgets.Button(description='Login')
logout_button = widgets.Button(description='Log out')

login_out = widgets.HTML()

login_success = False

def login(event):

    """ 
    
    Manage login event: 
    
    username: employee username
    password: employee password
    
    Return acknowledgement message
     
    """

    global base_url
    global login_success

    username = username_input.value
    password = password_input.value

    try:
        response = requests.post(base_url, json={'username': username, 'password': password})
        res = response.json()

        if response.status_code == 200: # Successful
            login_out.value = f'<div style="color: green;"> {res}</div>'
            login_success = True

        else:  # Failure
            login_out.value = f'<div style="color: red;"> {res}</div>'

    except requests.JSONDecodeError as e:
        login_out.value = f"<div style='text-align: center; color: red;'> Bad request: Backend error</div>"


def logout(event):
    
    global login_success
    
    login_success = False
    login_out.value = f'<div>Logged out.</div>'
    # stop_server()


# Page title
title = widgets.HTML(value="<h1 style='text-align: center;'>Data Brigade - Hotel System</h1>")

# Attach login function to login button click event
login_button.on_click(login)
logout_button.on_click(logout)

# Define section layout
log_buttons = widgets.HBox([login_button, logout_button])

login_box = widgets.HBox([title, username_input, password_input, log_buttons, login_out])
login_box.layout=widgets.Layout(flex_flow='column', align_items='center', height='auto', width='auto')

# display(login_box)

# tabs = widgets.Tab()
# tabs.children = [login_box]
# tab_titles = ['Login', 'Reservations', 'Analytics']
# 
# display(tabs)
    

In [464]:
# Local statistics

local_output = widgets.Output()

def handicap_rooms(hid):
    """ 
    
    Top 5 Handicap Rooms with Most Reservations: 
    If authorized, show graph. Otherwise, print unauthorization message. 
    
    """

    global base_url

    try:
        # fetch data
        response = requests.get(base_url + f'/hotel/{hid}/handicaproom')
        data = response.json()

        if response.status_code == 200: # successful fetch --> generate graph
            
            df = pd.DataFrame(data)

            fig, ax = plt.subplots()
            
            rooms = [f"Room {room}" for room in df['room']]
            res_count = df['reservation'].tolist()

            g = ax.bar(rooms, res_count)

            ax.set_ylabel('Reservations')
            ax.set_title('Top 5 Handicap Rooms with Most Reservations')
            ax.bar_label(g, res_count)

            plt.show()

        else:   # error
            with local_output:
                display(widgets.HTML(f'<div style="text-align: center; color: red;"> {data}</div>'))

    except requests.JSONDecodeError as e:
        with local_output:
            display(widgets.HTML(f"<div style='text-align: center; color: red;'> Bad request: Backend error</div>"))


def least_reserve(hid):
    """ 

    Top 3 rooms with the least time unavailable: 
    If authorized, show graph. Otherwise, print unauthorization message. 
    
    """

    global base_url

    try:

        # fetch data
        response = requests.get(base_url + f'/hotel/{hid}/leastreserve')
        data = response.json()

        if response.status_code == 200: # successful fetch --> generate graph
            
            df = pd.DataFrame(data)

            fig, ax = plt.subplots()

            rooms = [f"Room {room}" for room in df['room']]
            days = df['unavailable_days'].tolist()

            g = ax.bar(rooms, days)

            ax.set_ylabel('Days Unavailable')
            ax.set_title('Top 3 Rooms with Least Unavailable Time')
            ax.bar_label(g, days)

            plt.show()

        else:   # error 
            with local_output:
                display(widgets.HTML(f'<div style="text-align: center; color: red;"> {data}</div>'))
                
    except requests.JSONDecodeError as e:
        with local_output:
            display(widgets.HTML(f"<div style='text-align: center; color: red;'> Bad request: Backend error</div>"))


def most_creditcard(hid):
    """ 

    Top 5 clients under 30 that made the most reservation with a credit card. 
    If authorized, show graph. Otherwise, print unauthorization message. 

    """

    global base_url
    
    try:
        # fetch data
        response = requests.get(base_url+f'/hotel/{hid}/mostcreditcard')
        data = response.json()

        if response.status_code == 200: # successful fetch

            df = pd.DataFrame(data)

            fig, ax = plt.subplots()

            clids = [f'Client {id}' for id in df['client']]
            res_count = df["reservations"].tolist()

            g = ax.bar(clids, res_count)

            ax.set_ylabel('Reservations')
            ax.set_title('Top 5 clients under 30 that made the most reservation with a credit card.')
            ax.bar_label(g, res_count)

            # Display the figure widget with the desired layout
            with local_output:
                plt.tight_layout()
                plt.show()

        else:   # error 
            with local_output:
                display(widgets.HTML(f'<div style="text-align: center; color: red;"> {data}</div>'))
    
    except requests.JSONDecodeError as e:
        with local_output:
            display(widgets.HTML(f"<div style='text-align: center; color: red;'> Bad request: Backend error</div>"))

    
    
def highest_paid(hid):
    """ 
    
    Top 3 highest paid regular employees.
    If authorized, show graph. Otherwise, print unauthorization message.

    """

    global base_url

    try:
        # fetch data
        response = requests.get(base_url + f'/hotel/{hid}/highestpaid')
        data = response.json()

        if response.status_code == 200: # successful fetch

            df = pd.DataFrame(data)

            fig, ax = plt.subplots()

            employees = [f'Employee {id}' for id in df['employee']]
            salaries = [float(i) for i in df['salary']]

            g = ax.bar(employees, salaries)

            ax.set_ylabel('Salary ($)')
            ax.set_title('Top 3 highest paid regular employees.')
            ax.bar_label(g, [f'${sal}' for sal in salaries])

            plt.show()
        
        else:   # error 
            with local_output:
                display(widgets.HTML(f'<div style="text-align: center; color: red;"> {data}</div>'))

    except requests.JSONDecodeError as e:
        with local_output:
            display(widgets.HTML(f"<div style='text-align: center; color: red;'> Bad request: Backend error</div>"))



def most_discount(hid):
    """ 
    
    Top 3 highest paid regular employees.
    If authorized, show graph. Otherwise, print unauthorization message.

    """

    global base_url

    try:
        # fetch data
        response = requests.get(base_url + f'/hotel/{hid}/mostdiscount')
        data = response.json()

        if response.status_code == 200: # successful fetch

            df = pd.DataFrame(data)

            fig, ax = plt.subplots()

            clids = [f'Client {id}' for id in df['clid']]
            discount = [float(i) for i in df['discount_amount']]

            g = ax.bar(clids, discount)

            ax.set_ylabel('Discounted Amount ($)')
            ax.set_title('Top 5 clients that received the most discounts.')
            ax.bar_label(g, [f'${amount}' for amount in discount])

            plt.show()

        else:   # error
            with local_output:
                display(widgets.HTML(f'<div style="text-align: center; color: red;"> {data}</div>'))

    except requests.JSONDecodeError as e:
        with local_output:
           display(widgets.HTML(f"<div style='text-align: center; color: red;'> Bad request: Backend error</div>"))


def room_type(hid):
    """ 
    
    Total reservation by room type
    If authorized, show graph. Otherwise, print unauthorization message.

    """

    global base_url

    try:
        # fetch data
        response = requests.get(base_url + f'/hotel/{hid}/roomtype')
        data = response.json()

        if response.status_code == 200:
            
            df = pd.DataFrame(data)

            fig, ax = plt.subplots()

            rtype = df['room_type']
            res_count = [int(i) for i in df['reservations']]

            g = ax.bar(rtype, res_count)

            ax.set_ylabel('Reservations')
            ax.set_title('Total reservation by room type')
            ax.bar_label(g, res_count)

            plt.show()

        else:   # error
            with local_output:
                display(widgets.HTML(f'<div style="text-align: center; color: red;"> {data}</div>'))
    
    except requests.JSONDecodeError as e:
        with local_output:
            display(widgets.HTML(f"<div style='text-align: center; color: red;'> Bad request: Backend error</div>"))

    
    
def least_guests(hid):
    """ 
    
    Top 3 rooms that were reserved that had the least guest-to-capacity ratio.
    If authorized, show graph. Otherwise, print unauthorization message.

    """

    global base_url

    try:
        response = requests.get(base_url + f'/hotel/{hid}/leastguests')
        data = response.json()

        if response.status_code == 200:

            df = pd.DataFrame(data)

            fig, ax = plt.subplots()

            rids = [f'Room {id}' for id in df['rid']]
            ratios = [round(float(i), 2) for i in df['ratio']]

            g = ax.bar(rids, ratios)

            ax.set_ylabel('Ratio')
            ax.set_title('Top 3 rooms that were reserved that had the least guest-to-capacity ratio.')
            ax.bar_label(g, ratios)

            plt.show()

        else:   # error
            with local_output:
                display(widgets.HTML(f'<div style="text-align: center; color: red;"> {data}</div>'))
                
    except requests.JSONDecodeError as e:
        with local_output:
            display(widgets.HTML(f"<div style='text-align: center; color: red;'> Bad request: Backend error</div>"))


# input
# stat selection
local_selection = widgets.Dropdown(
    # TODO: Add other stats
    options=['Choose', 'Handicap rooms', 
             'Least reserve', 'Most credit card',
             'Highest paid', 'Most discount',
             'Room type', 'Least guests'],
    value='Choose',
    description='Statistic: '
)

# hid input
local_hid = widgets.BoundedIntText(
    value=1,
    min=1,
    max=100000000,
    description='Hotel ID: '
)

def local_display(event):
    hid = local_hid.value
    with local_output:
        local_output.clear_output(wait=False)
        match local_selection.value:
            case 'Choose':
                local_output.clear_output(wait=False)
            case 'Handicap rooms':
                local_output.value = handicap_rooms(hid)
            case 'Least reserve':
                local_output.value = least_reserve(hid)
            case 'Most credit card':
                local_output.value = most_creditcard(hid)
            case 'Highest paid':
                local_output.value = highest_paid(hid)
            case 'Most discount':
                local_output.value = most_discount(hid)
            case 'Room type':
                local_output.value = room_type(hid)
            case 'Least guests':
                local_output.value = least_guests(hid)

local_button = widgets.Button(description='Submit')

local_button.on_click(local_display)

# styling & layout

local_title = widgets.HTML('<h2>Local Statistics<h2>')

local_in = widgets.HBox([local_selection, local_hid, local_button])
local_in.layout=widgets.Layout(flex_flow='column', align_items='center')

local_header = widgets.HBox([local_title, local_in])
local_header.layout=widgets.Layout(flex_flow='column', align_items='center')

local_stats = widgets.AppLayout(
    header = local_header, 
    center = local_output
)

In [465]:
# Global statistics

global_output = widgets.Output()

def most_revenue():
        """ 
    
        Top 5 hotel chains with the highest total revenue:
        If authorized, show graph. Otherwise, print unauthorization message. 
        
        """

        global base_url

        try:
            # fetch data
            response = requests.get(base_url + '/most/revenue')
            data = response.json()
            if response.status_code == 200:
                df = pd.DataFrame(data)

                # generate graph
                chains = df['chain_name']
                totals = df['total_revenue']
                g = plt.bar(chains, totals)
                plt.bar_label(g, labels=totals)
                plt.xlabel('Chains')
                plt.ylabel('Revenues')
                plt.title('Top 5 hotel chains with the highest total revenue')

                plt.show()
            else: # error code
                with global_output:
                    display(widgets.HTML(f'<div style="color: red;"> {data}</div>'))


        except requests.JSONDecodeError as e:
            with global_output:
                display(widgets.HTML(f"<div style='text-align: center; color: red;'> Bad request: Backend error</div>"))


def payment_method():
        """ 
    
        Total reservation percentage by payment method:
        If authorized, show graph. Otherwise, print unauthorization message. 
        
        """

        global base_url

        try:
            # fetch data
            response = requests.get(base_url + '/paymentmethod')
            data = response.json()
            if response.status_code == 200:
                df = pd.DataFrame(data)

                # generate graph
                payments = df['payment']
                reservations = df['reservation_percentage']
                g = plt.pie(reservations, labels=payments, autopct='%1.1f%%')
                plt.show()

        except requests.JSONDecodeError as e:
            with global_output:
                display(f'<div style="color: red;"> Bad request: Backend error</div>')

def least_rooms():
        """

        Top 3 chains with the least rooms:
        If authorized, show graph. Otherwise, print unauthorization message.

        """

        global base_url

        try:
            # fetch data
            response = requests.get(base_url + '/least/rooms')
            data = response.json()
            if response.status_code == 200:
                df = pd.DataFrame(data)

                # generate graph
                name = df['chain_name']
                rooms = df['total_rooms']
                g = plt.bar(name, rooms)
                plt.bar_label(g, labels=rooms)
                plt.xlabel('Chains')
                plt.ylabel('Rooms')
                plt.title('Top 3 Hotel Chains With The Least Rooms')

                plt.show()
            else:
                with global_output:
                    display(widgets.HTML(f'<div style="color: red;"> {data}</div>'))

        except requests.JSONDecodeError as e:
            with global_output:
                display(f'<div style="color: red;"> Bad request: Backend error</div>')

def most_capacity():
        """

        Top 5 hotels with the most capacity:
        If authorized, show graph. Otherwise, print unauthorization message.

        """

        global base_url

        try:
            # fetch data
            response = requests.get(base_url + '/most/capacity')
            data = response.json()
            if response.status_code == 200:
                df = pd.DataFrame(data)

                # generate graph
                name = df['hotel_name']
                capacity = df['capacity']
                g = plt.bar(name, capacity)
                plt.bar_label(g, labels=capacity)
                plt.xlabel('Hotels')
                plt.ylabel('Capacity')
                plt.title('Top 5 Hotels With The Most Capacity')
                plt.xticks(rotation=45, ha='right')

                plt.show()
            else:
                with global_output:
                    display(widgets.HTML(f'<div style="color: red;"> {data}</div>'))

        except requests.JSONDecodeError as e:
            with global_output:
                display(widgets.HTML(f"<div style='text-align: center; color: red;'> Bad request: Backend error</div>"))

def most_reservations():
        """

        Top 10% of hotels with the most reservations:
        If authorized, show graph. Otherwise, print unauthorization message.

        """

        global base_url

        try:
            # fetch data
            response = requests.get(base_url + '/most/reservation')
            data = response.json()
            if response.status_code == 200:
                df = pd.DataFrame(data)

                # generate graph
                name = df['hotel_name']
                reservations = df['most_reservations']
                g = plt.bar(name, reservations)
                plt.bar_label(g, labels=reservations)
                plt.xlabel('Hotels')
                plt.ylabel('Reservations')
                plt.title('Top 10% of Hotels with the Most Reservations')
                plt.xticks(rotation=45, ha='right')

                plt.show()
            else:
                with global_output:
                    display(widgets.HTML(f'<div style="color: red;"> {data}</div>'))

        except requests.JSONDecodeError as e:
            with global_output:
                display(f'<div style="color: red;"> Bad request: Backend error</div>')

def most_profitable():
        """

        Top 3 months with the most profitable months of all chains
        If authorized, show graph. Otherwise, print unauthorization message.

        """

        global base_url

        try:
            # fetch data
            response = requests.get(base_url + '/most/profitmonth')
            data = response.json()
            if response.status_code == 200:
                df = pd.DataFrame(data)

                # generate graph
                pivot_df = df.pivot(columns='chain_name', values='reservations_count')
                bars = pivot_df.plot(kind='bar', stacked=False, width=1)

                # Plot customization
                plt.xlabel('Months')
                plt.ylabel('Reservation Count')
                plt.title('Reservation Counts for Each Chain Across Top 3 Most Profitable Months')

                months = df['months']
                num_chains = len(months)
                plt.xticks(np.arange(num_chains), months, rotation=90, ha='center')

                plt.legend(title='Chain', bbox_to_anchor=(1.05, 1), loc='upper left')

                for bar in bars.containers:
                    for rect in bar:
                        height = rect.get_height()
                        if height > 0:
                            plt.text(rect.get_x() + rect.get_width() / 2.0, height, '%d' % int(height), ha='center', va='bottom')

                plt.show()
            else:
                with global_output:
                    display(widgets.HTML(f'<div style="color: red;"> {data}</div>'))

        except requests.JSONDecodeError as e:
            with global_output:
                display(f'<div style="color: red;"> Bad request: Backend error</div>')

# input
# stat selection
global_selection = widgets.Dropdown(
    # TODO: Add other stats
    options=['Choose', 'Most Revenue', 'Payment Method', 'Least Rooms', 'Most Capacity', 'Most Reservation', 'Most Profitable Month'],
    value='Choose',
    description='Statistic: '
)

global_output.layout = widgets.Layout(display='flex', justify_content='center', align_items='center', height='500px')

def global_display(event):
    # TODO: Add other stats

    with global_output:
        global_output.clear_output(wait=True)
        match global_selection.value:
            case 'Choose':
                global_output.clear_output(wait=False)
            case 'Most Revenue':
                global_output.value = most_revenue()
            case 'Payment Method' :
                global_output.value = payment_method()
            case 'Least Rooms':
                global_output.value = least_rooms()
            case 'Most Capacity':
                global_output.value = most_capacity()
            case 'Most Reservation':
                global_output.value = most_reservations()
            case 'Most Profitable Month':
                global_output.value = most_profitable()


global_button = widgets.Button(description='Submit')
global_button.on_click(global_display)

# styling & layout

global_title = widgets.HTML('<h2>Global Statistics<h2>')

global_input = widgets.HBox([global_selection, global_button])

global_header = widgets.HBox([global_title, global_input])
global_header.layout=widgets.Layout(flex_flow='column', align_items='center')

global_stats = widgets.AppLayout(
    header = global_header, 
    center = global_output
)  

In [466]:
# Room unavailable

# input
room_id = widgets.IntText(
    value=1,
    description='Room ID: '
)

startdate_in = widgets.DatePicker(
    description='Start date: '
)

enddate_in = widgets.DatePicker(
    description='End date: '
)

startyear = None
startmonth = None
startday = None

def on_start_change(change):
    global startyear, startmonth, startday
    selected_date = change['new']
    startyear = selected_date.year
    startmonth = selected_date.month
    startday = selected_date.day

endyear = None
endmonth = None
endday = None

def on_end_change(change):
    global endyear, endmonth, endday
    selected_date = change['new']
    endyear = selected_date.year
    endmonth = selected_date.month
    endday= selected_date.day

startdate_in.observe(on_start_change, names='value')
enddate_in.observe(on_end_change, names='value')

# Outputs
description_out = widgets.HTML()
unavailable_out = widgets.HTML()

# Handle fetch
def get_roomdescription(event):
    """ 
    See room details.

        rid : (room ID)

    Returns acknowledgement message. 
    
    """

    global base_url

    try:
        rid = room_id.value

        # Access room
        room_res = requests.get(base_url + f'/room/{rid}')
        room_json = room_res.json()

        if room_res.status_code == 200:
            rdid = int(room_json['rdid'])

            # Manage room description
            description_res = requests.get(base_url + f'/roomdescription/{rdid}')
            description_json = description_res.json()

            if description_res.status_code == 200:
                rname = f'<b>Name: </b><i>{description_json['rname']}</i>'
                rtype = f'<b>Type: </b><i>{description_json['rtype']}</i>'
                capacity = f'<b>Capacity: </b><i>{description_json['capacity']}</i>'
                ishandicap = f'<b>Handicap: </b><i>{description_json['ishandicap']}</i>'
                description_out.value = f'<h1>Room No. {rid} Details:</h1><pre>{rname}\n{rtype}\n{capacity}\n{ishandicap}</pre>'

            else:
                description_out.value = f'<p style="color: red;">{description_json}</p>'
        
        else:
            description_out.value = f'<p style="color: red;">{room_json}</p>'
    
    except requests.JSONDecodeError as e:
        description_out.value = f"<div style='text-align: center; color: red;'> Bad request: Backend error</div>"
    

# Handle insertion
def post_roomunavailable(event):

    """ 
    Create a new roomunavailable record.

        rid : (room ID) Room to be unavailable from startdate to enddate.
        startdate : First day of reservation
        enddate : Last day of reservation

    Returns acknowledgement message. 
    
    """
    
    global base_url
    global startyear, startmonth, startday
    global endyear, endmonth, endday

    # Input
    rid = room_id.value
    startdate = f'{startyear}-{startmonth}-{startday}'
    enddate = f'{endyear}-{endmonth}-{endday}'

    try:
        response = requests.post(base_url+'/roomunavailable', json={'rid': rid, 'startdate': startdate, 'enddate': enddate})
        res = response.json()
        
        if response.status_code == 201: # Successful
            unavailable_out.value = f'<h1>Room No. {res['rid']} Separation ID: {res['ruid']}</h1><pre><b>From: </b><i>{res['startdate']}</i>\n<b>To: </b><i>{res['enddate']}</i></pre>'

        else: # Failure
            unavailable_out.value = f'<p style="color: red;">{res}</p>'
            
    except requests.JSONDecodeError as e:
        unavailable_out.value = f"<div style='text-align: center; color: red;'> Bad request: Backend error</div>"

create_roomunavailable = widgets.Button(
    description='Separate room'
)

check_room = widgets.Button(
    description='Get room details'
)

create_roomunavailable.on_click(post_roomunavailable)
check_room.on_click(get_roomdescription)

# styling & layout

# Room ID input + Room details button
ru_title = widgets.HTML("<h2 style='text-align: center;'>New room unavailable period<h2>")

check_room_input = widgets.HBox([room_id, check_room])
check_room_box = widgets.HBox([check_room_input, description_out])
dates_box = widgets.HBox([startdate_in, enddate_in, create_roomunavailable, unavailable_out])

check_room_input.layout=widgets.Layout(flex_flow='column', align_items='center')
check_room_box.layout=widgets.Layout(flex_flow='column', align_items='center')
dates_box.layout=widgets.Layout(flex_flow='column', align_items='center')

ru_page = widgets.AppLayout(
    header = ru_title,
    center = widgets.TwoByTwoLayout(top_left=check_room_box, top_right=dates_box)
)

In [467]:
# Create a reservation

# Input
ruid_in = widgets.BoundedIntText(
    description='Separation ID: ',
    value=1,
    min=1,
    max=10000000000,
    disabled=False,
    step=1
)

client_in = widgets.BoundedIntText(
    description='Client ID: ',
    value=1,
    min=1,
    max=10000000000,
    disabled=False,
    step=1
)

guests_in = widgets.BoundedIntText(
    description='Guests: ',
    value=1,
    min=1,
    max=8,
    disabled=False,
    step=1
)

payment_in = widgets.Dropdown(
    description='Payment: ',
    options=['cash', 'check', 'credit card', 'pear pay', 'debit card'],
    value='cash'
)

# Output
res_out = widgets.HTML()

# Event handling
def post_reservation(event):

    global base_url

    ruid = ruid_in.value
    clid = client_in.value
    guests = guests_in.value
    payment = guests_in.value

    try:

        response = requests.post(base_url + '/reserve', json={'ruid': ruid, 'clid': clid, 'guests': guests, 'payment': payment})
        data = response.json()

        if response.status_code == 200:
            header = f'<h3 style="text-align: center;">Confirmation -  <i>Reservation ID: {data['reid']}</i></h3>'
            res_out.value = header + f'<pre style="text-align: center;"><b>Total: </b><i>${data['total_cost']}</i></pre>'

        else:
            res_out.value = f"<div style='text-align: center; color: red;'>{data}</div>"

    except requests.JSONDecodeError as e:
        res_out.value = f"<div style='text-align: center; color: red;'> Bad request: Backend error</div>"
    
res_button = widgets.Button(
    description='Create reservation'
)

res_button.on_click(post_reservation)

# Styling and layout

res_title = widgets.HTML("<h2 style='text-align: center;'>New reservation information<h2>")

ruid_in.style.description_width='auto'
client_in.style.description_width='auto'
guests_in.style.description_width='auto'
payment_in.style.description_width='auto'

ruid_in.layout.align_self='center'
client_in.layout.align_self='center'
guests_in.layout.align_self='center'
payment_in.layout.align_self='center'

res_header = widgets.HBox([res_title, ruid_in, client_in, guests_in, payment_in, res_button])
res_header.layout.flex_flow='column'
res_header.layout.align_items='center'

res_page = widgets.AppLayout(header = res_header, center = res_out)

In [468]:
# Add a client

# Input
fname_in = widgets.Text(
    description='First name: '
)

lname_in = widgets.Text(
    description='Last name: '
)

age_in = widgets.IntText(
    description='Age: ',
)

memberyear_in = widgets.IntText(
    description='Member years: ',
)

# Output
client_out = widgets.HTML()

# Event handling
def post_client(event):

    global base_url

    fname = fname_in.value
    lname = lname_in.value
    age = age_in.value
    memberyear =memberyear_in.value

    try:
        response = requests.post(base_url + '/client', json={'fname': fname, 'lname': lname, 'age': age, 'memberyear': memberyear})
        data = response.json()

        if response.status_code == 200:
            # confirmation
            client_out.value = f'<h3 style="text-align: center;">Confirmation -  <i>Client ID: {data['clid']}</i></h3>'
        else:
            # error message
            client_out.value = f"<div style='text-align: center; color: red;'>{data}</div>"
            
    except requests.JSONDecodeError as e:
        client_out.value = f"<div style='text-align: center; color: red;'> Bad request: Backend error</div>"

client_button = widgets.Button(
    description='Create client'
)

client_button.on_click(post_client)
client_button.layout.align_self='center'

# Styling & layout

client_title = widgets.HTML('<h2 style="text-align: center;">New client information</h2>')

fname_in.style.description_width='auto'
lname_in.style.description_width='auto'
age_in.style.description_width='auto'
memberyear_in.style.description_width='auto'

fname_in.layout.align_self='center'
lname_in.layout.align_self='center'
age_in.layout.align_self='center'
memberyear_in.layout.align_self='center'

client_in_box = widgets.HBox([client_title, fname_in, lname_in, age_in, memberyear_in, client_button])
client_in_box.layout = widgets.Layout(flex_flow='column', justify_content='center')

client_page = widgets.AppLayout(header=client_in_box, center=client_out)

In [469]:
# Add a room description

# input widgets
rname_in = widgets.Dropdown(
    description='Room name: ',
    options=['Standard', 'Standard Queen', 'Standard King', 
               'Double Queen', 'Double King', 'Triple King'
               'Executive Family', 'Presidential']
)

rtype_in = widgets.Dropdown(
    description='Room type: ',
    options=['Basic', 'Premium', 'Deluxe', 'Suite']
)

capacity_in = widgets.BoundedIntText(
    description='Capacity: ',
    min=1,
    max=8
)

ishandicap_in = widgets.Dropdown(
    description='Handicap: ',
    options=[('Yes', True), ('No', False)]
)

# output widget
rd_out = widgets.HTML()

# handle insertion
def post_rd(event):

    global base_url

    rname = rname_in.value
    rtype = rtype_in.value
    capacity = capacity_in.value
    ishandicap = ishandicap_in.value

    try:
        response = requests.post(base_url + '/roomdescription', json={'rname': rname, 'rtype': rtype, 'capacity': capacity, 'ishandicap': ishandicap})
        data = response.json()

        if response.status_code == 200:
            rd_out.value = f'<h3 style="text-align: center;">Confirmation -  <i>Room description ID: {data['rdid']}</i></h3>'
        else:
            rd_out.value = f"<div style='text-align: center; color: red;'>{data}</div>"
            
    except requests.JSONDecodeError as e:
        rd_out.value = f"<div style='text-align: center; color: red;'> Bad request: Backend error</div>"


# buttons
rd_button = widgets.Button(
    description='Submit'
)

rd_button.on_click(post_rd)
rd_button.layout.align_self='center'

# styling & layout
rd_title = widgets.HTML('<h2 style="text-align: center;">New Room Description</h2>')

rname_in.style.description_width='auto'
rtype_in.style.description_width='auto'
capacity_in.style.description_width='auto'
ishandicap_in.style.description_width='auto'

rname_in.layout.align_self='center'
rtype_in.layout.align_self='center'
capacity_in.layout.align_self='center'
ishandicap_in.layout.align_self='center'

rd_header = widgets.HBox([rd_title, rname_in, rtype_in, capacity_in, ishandicap_in, rd_button])
rd_header.layout = widgets.Layout(flex_flow='column', justify_content='center')

rd_page = widgets.AppLayout(header=rd_header, center=rd_out)

In [470]:
# Add a room

# Input
hid_in = widgets.BoundedIntText(
    description='Hotel ID ',
    min=1,
    max=10000000000
)

rdid_in = widgets.BoundedIntText(
    description='Room description ID: ',
    min=1,
    max=1000000000
)

rprice_in = widgets.BoundedFloatText(
    description='Room price: ',
    min=0.0,
    max=1000000000,
    step=0.1
)

# Output
room_out = widgets.HTML()

# Event handling
def post_room(event):

    global base_url

    hid = hid_in.value
    rdid = rdid_in.value
    rprice = rprice_in.value

    try:
        response = requests.post(base_url + '/room', json={'hid': hid, 'rdid': rdid, 'rprice': rprice})
        data = response.json()

        if response.status_code == 200:
            # confirmation
            room_out.value = f'<h3 style="text-align: center;">Confirmation -  <i>Room ID: {data['rid']}</i></h3>'
        else:
            # error message
            room_out.value = f"<div style='text-align: center; color: red;'>{data}</div>"
            
    except requests.JSONDecodeError as e:
        room_out.value = f"<div style='text-align: center; color: red;'> Bad request: Backend error</div>"

room_button = widgets.Button(description='Create room')

room_button.on_click(post_room)
room_button.layout.align_self='center'

# Styling & layout

room_title = widgets.HTML('<h2 style="text-align: center;">New Room Information</h2>')

hid_in.style.description_width='auto'
rdid_in.style.description_width='auto'
rprice_in.style.description_width='auto'

hid_in.layout.align_self='center'
rdid_in.layout.align_self='center'
rprice_in.layout.align_self='center'

room_header = widgets.HBox([room_title, rdid_in, rprice_in, room_button])
room_header.layout = widgets.Layout(flex_flow='column', justify_content='center')

room_page = widgets.AppLayout(header=room_header, center=room_out)

In [471]:
# Add a hotel

# Input
chid_in = widgets.BoundedIntText(
    description='Chain ID: ',
    min=1,
    max=1000000000
)

hname_in = widgets.Text(
    description='Hotel name: '
)

hcity_in = widgets.Text(
    description='Hotel city: '
)

# Output
hotel_out = widgets.HTML()

# Event handling
def post_hotel(event):

    global base_url

    chid = chid_in.value
    hname = hname_in.value
    hcity = hcity_in.value

    try:
        response = requests.post(base_url + '/hotel', json={'chid': chid, 'hname': hname, 'hcity': hcity})
        data = response.json()

        if response.status_code == 200:
            # confirmation
            hotel_out.value = f'<h3 style="text-align: center;">Confirmation -  <i>Hotel ID: {data['hid']}</i></h3>'
        else:
            # error message
            hotel_out.value = f"<div style='text-align: center; color: red;'>{data}</div>"
            
    except requests.JSONDecodeError as e:
        hotel_out.value = f"<div style='text-align: center; color: red;'> Bad request: Backend error</div>"

hotel_button = widgets.Button(
    description='Create hotel'
)

hotel_button.on_click(post_hotel)
hotel_button.layout.align_self='center'

# Styling & layout

hotel_title = widgets.HTML('<h2 style="text-align: center;">New Hotel Information</h2>')

chid_in.style.description_width='auto'
hname_in.style.description_width='auto'
hcity_in.style.description_width='auto'

chid_in.layout.align_self='center'
hname_in.layout.align_self='center'
hcity_in.layout.align_self='center'

hotel_header = widgets.HBox([hotel_title, chid_in, hname_in, hcity_in, hotel_button])
hotel_header.layout = widgets.Layout(flex_flow='column', justify_content='center')

hotel_page = widgets.AppLayout(header=hotel_header, center=hotel_out)

In [472]:
# Add a chain

# Input
post_cname_in = widgets.Text(description='Chain name: ')
post_springmkup_in = widgets.BoundedFloatText(description='Spring markup: ')
post_summermkup_in = widgets.BoundedFloatText(description='Summer markup: ')
post_fallmkup_in = widgets.BoundedFloatText(description='Fall markup: ')
post_wintermkup_in = widgets.BoundedFloatText(description='Winter markup: ')

# Output
chain_out = widgets.HTML()

def post_chain(event):

    global base_url

    cname = post_cname_in.value
    springmkup = post_springmkup_in.value
    summermkup = post_summermkup_in.value
    fallmkup = post_fallmkup_in.value
    wintermkup = post_wintermkup_in.value

    try:
        response = requests.post(base_url + '/chains', 
                                json={'cname': cname, 'springmkup': springmkup, 
                                    'summermkup': summermkup, 'fallmkup': fallmkup, 
                                    'wintermkup': wintermkup})
        data = response.json()

        if response.status_code == 200:
            # confirmation
            chain_out.value = f'<h3 style="text-align: center;">Confirmation -  <i>Chain ID: {data['chid']}</i></h3>'
        else:
            # error message
            chain_out.value = f"<div style='text-align: center; color: red;'>{data}</div>"

    except requests.JSONDecodeError as e:
        chain_out.value = f"<div style='text-align: center; color: red;'> Bad request: Backend error</div>"


create_chain_button = widgets.Button(description='Create chain')

create_chain_button.on_click(post_chain)

create_chain_button.layout.align_self='center'

# Styling & layout

post_chain_title = widgets.HTML('<h2 style="text-align: center;">New Chain Information</h2>')

post_cname_in.style.description_width='auto'
post_springmkup_in.style.description_width='auto'
post_summermkup_in.style.description_width='auto'
post_fallmkup_in.style.description_width='auto'
post_wintermkup_in.style.description_width='auto'

post_cname_in.layout.align_self='center'
post_springmkup_in.layout.align_self='center'
post_summermkup_in.layout.align_self='center'
post_fallmkup_in.layout.align_self='center'
post_wintermkup_in.layout.align_self='center'

post_chain_header = widgets.HBox([post_chain_title, post_cname_in, post_springmkup_in, post_summermkup_in, post_fallmkup_in, post_wintermkup_in, create_chain_button])
post_chain_header.layout = widgets.Layout(flex_flow='column', justify_content='center')

post_chain_section = widgets.AppLayout(header=post_chain_header, center=chain_out)

chain_page = widgets.HBox([post_chain_section])
chain_page.layout = widgets.Layout(flex_flow='column', justify_content='center')

In [473]:

e_hid_in = widgets.BoundedIntText(description='Hotel ID: ', min=1, max=1000000000)
e_fname_in = widgets.Text(description='First name: ')
e_lname_in = widgets.Text(description='Last name: ')
e_position_in = widgets.RadioButtons(
    options=['Regular', 'Supervisor', 'Administrator'],
    description='Position: '
)
e_salary_in = widgets.BoundedFloatText(description='Salary: ', min=1, max=10000000000)
e_age_in = widgets.BoundedIntText(description='Age: ', min=1, max=1000000000)

em_out = widgets.HTML()

def post_employee(event):

    global base_url

    e_hid = e_hid_in.value
    e_fname = e_fname_in.value
    e_lname = e_lname_in.value
    e_position = e_position_in.value
    e_salary = e_salary_in.value
    e_age = e_age_in.value

    try:
        response = requests.post(base_url + '/employee',
                                json={'hid': e_hid, 'fname': e_fname,
                                    'lname': e_lname, 'position': e_position,
                                    'salary': e_salary, 'age': e_age})
        data = response.json()

        if response.status_code == 200:
            # confirmation
            em_out.value = f'<h3 style="text-align: center;">Confirmation -  <i>Employee ID: {data['eid']}</i></h3>'
        else:
            # error message
            em_out.value = f"<div style='text-align: center; color: red;'>{data}</div>"

    except requests.JSONDecodeError as e:
        em_out.value = f"<div style='text-align: center; color: red;'> Bad request: Backend error</div>"

create_employee = widgets.Button(description='Add employee')

create_employee.on_click(post_employee)

create_employee.layout.align_self='center'

em_title = widgets.HTML('<h2 style="text-align: center;">New Employee Information</h2>')

e_hid_in.style.description_width='auto'
e_fname_in.style.description_width='auto'
e_lname_in.style.description_width='auto'
e_salary_in.style.description_width='auto'
e_position_in.style.description_width='auto'
e_age_in.style.description_width='auto'

e_hid_in.layout.align_self='center'
e_fname_in.layout.align_self='center'
e_lname_in.layout.align_self='center'
e_salary_in.layout.align_self='center'
e_position_in.layout.align_self='center'
e_age_in.layout.align_self='center'

em_header = widgets.HBox([em_title, e_hid_in, e_fname_in, e_lname_in, e_salary_in, e_position_in, e_age_in, create_employee])
em_header.layout = widgets.Layout(flex_flow='column', justify_content='center')

em_page = widgets.AppLayout(header=em_header, center=em_out)

In [474]:
eid_in = widgets.BoundedIntText(description='Employee ID: ', min=1, max=100000000)
user_in = widgets.Text(description='Username: ')
pass_in = widgets.Password(description='Password: ')

l_out = widgets.HTML()

def post_login(event):

    global base_url

    eid = eid_in.value
    user = user_in.value
    pas = pass_in.value

    try:
        response = requests.post(base_url + '/login',
                                json={'eid': eid, 'username': user,
                                    'password': pas})
        data = response.json()

        if response.status_code == 200:
            # confirmation
            l_out.value = f'<h3 style="text-align: center;">Confirmation -  <i>Login ID: {data['lid']}</i></h3>'
        else:
            # error message
            l_out.value = f"<div style='text-align: center; color: red;'>{data}</div>"

    except requests.JSONDecodeError as e:
        l_out.value = f"<div style='text-align: center; color: red;'> Bad request: Backend error</div>"

create_login = widgets.Button(description='Create login')

create_login.on_click(post_login)

create_login.layout.align_self='center'

l_title = widgets.HTML('<h2 style="text-align: center;">New Login Credentials</h2>')

eid_in.style.description_width='auto'
user_in.style.description_width='auto'
pass_in.style.description_width='auto'

eid_in.layout.align_self='center'
user_in.layout.align_self='center'
pass_in.layout.align_self='center'

l_header = widgets.HBox([l_title, eid_in, user_in, pass_in, create_login])
l_header.layout = widgets.Layout(flex_flow='column', justify_content='center')

l_page = widgets.AppLayout(header=l_header, center=l_out)

In [475]:
setting_out = widgets.Output()

setting = widgets.ToggleButtons(
    options=['Unavailability', 'Reservation', 'Client'],
    value='Unavailability'
)

def on_setting_change(change):
    with setting_out:
        clear_output(wait=True)
        if change['type'] == 'change' and change['name'] == 'value':
            if change['new'] == 'Unavailability':
                display(ru_page)
            elif change['new'] == 'Reservation':
                display(res_page)
            elif change['new'] == 'Client':
                display(client_page)

with setting_out:
    display(ru_page)

setting.observe(on_setting_change, names='value')

setting_box = widgets.HBox([setting, setting_out])
setting_box.layout=widgets.Layout(flex_flow='column', align_items='center')

left_box = widgets.HBox([login_box, setting_box])
left_box.layout=widgets.Layout(flex_flow='column', align_items='center')

In [476]:
admin_out = widgets.Output()

admin_setting = widgets.ToggleButtons(
    options=['Manage room descriptions', 'Manage rooms', 'Manage hotels', 'Manage chains', 'Manage employees', 'Manage logins'],
    tooltips=['Manage room descriptions', 'Manage rooms', 'Manage hotels', 'Manage chains', 'Manage employees', 'Manage logins'],
    value='Manage rooms'
)

def on_admin_change(change):
    with admin_out:
        clear_output(wait=True)
        if change['type'] == 'change' and change['name'] == 'value':
            if change['new'] == 'Manage room descriptions':
                display(rd_page)
            elif change['new'] == 'Manage rooms':
                display(room_page)
            elif change['new'] == 'Manage hotels':
                display(hotel_page)
            elif change['new'] == 'Manage chains':
                display(chain_page)
            elif change['new'] == 'Manage employees':
                display(em_page)
            elif change['new'] == 'Manage logins':
                display(l_page)

with admin_out:
    display(room_page)

admin_setting.observe(on_admin_change, names='value')

admin_box = widgets.HBox([admin_setting, admin_out])
admin_box.layout=widgets.Layout(flex_flow='column', justify_content='center')

admin_box_tabs = widgets.Tab()
admin_box_tabs.children = [rd_page, room_page, hotel_page, chain_page, em_page, l_page]
admin_box_tabs.titles = ['Room Descriptions', 'Room Information', 'Hotels', 'Chains', 'Employee', 'Login']

In [477]:
accordion = widgets.Accordion()
accordion.children=[local_stats, global_stats, admin_box_tabs]
accordion.titles=('Local statistics', 'Global statistics', 'Administration')
accordion.selected_index=2

make_a_res_tabs = widgets.Tab()
make_a_res_tabs.children = [ru_page, res_page, client_page]
make_a_res_tabs.titles = ['Unavailability', 'Reservations', 'Clients']

tabs = widgets.Tab()
tabs.children = [login_box, make_a_res_tabs, accordion]
tabs.titles = ['Login', 'Make A Reservation', 'Analytics']

# page = widgets.TwoByTwoLayout(top_left=left_box, top_right=accordion)
page = widgets.Box([tabs])
page.layout=widgets.Layout(flex_flow='column', align_items='center')

display(page)

Box(children=(Tab(children=(HBox(children=(HTML(value="<h1 style='text-align: center;'>Data Brigade - Hotel Sy…